In [12]:
import csv

import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split


In [13]:
dataset = 'model/handClassifier/keypoint.csv'
model_save_path = 'model/handClassifier/keypoint_classifier.keras'
tflite_save_path = 'model/handClassifier/keypoint_classifier.tflite'

# Set number of classes

In [14]:
NUM_CLASSES = 6

# Dataset reading

In [15]:
X_dataset = np.loadtxt(dataset, delimiter=',', dtype='float32', usecols=list(range(1, (21 * 2) + 1)))

In [16]:
y_dataset = np.loadtxt(dataset, delimiter=',', dtype='int32', usecols=(0))

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X_dataset, y_dataset, train_size=0.75, random_state=42)

# Model building

In [18]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input((21 * 2, )),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
])

In [19]:
model.summary()  # tf.keras.utils.plot_model(model, show_shapes=True)

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dropout_2 (Dropout)             │ (None, 42)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 20)             │           860 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 20)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 10)             │           210 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 6)              │            66 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,136 (4.44 KB)

 Trainable params: 1,136 (4.44 KB)

 Non-trainable params: 0 (0.00 B)

In [20]:
# Model checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    model_save_path, verbose=1, save_weights_only=False)
# Callback for early stopping
es_callback = tf.keras.callbacks.EarlyStopping(patience=20, verbose=1)

In [21]:
# Model compilation
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Model training

In [22]:
model.fit(
    X_train,
    y_train,
    epochs=1000,
    batch_size=128,
    validation_data=(X_test, y_test),
    callbacks=[cp_callback, es_callback]
)

Epoch 1/1000
 1/31 ━━━━━━━━━━━━━━━━━━━━ 23s 770ms/step - accuracy: 0.1172 - loss: 2.0581
Epoch 1: saving model to model/handClassifier/keypoint_classifier.keras
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.1544 - loss: 1.9313 - val_accuracy: 0.2407 - val_loss: 1.6538
Epoch 2/1000
 1/31 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.2266 - loss: 1.7050
Epoch 2: saving model to model/handClassifier/keypoint_classifier.keras
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2396 - loss: 1.6712 - val_accuracy: 0.3467 - val_loss: 1.5090
Epoch 3/1000
 1/31 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.3047 - loss: 1.5007
Epoch 3: saving model to model/handClassifier/keypoint_classifier.keras
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3132 - loss: 1.5372 - val_accuracy: 0.3576 - val_loss: 1.4149
Epoch 4/1000
 1/31 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.3594 - loss: 1.3691
Epoch 4: saving model to model/handClassifier/keypoint_classifier.keras
31/31 ━━━━━━━━━━━

In [23]:
# Model evaluation
val_loss, val_acc = model.evaluate(X_test, y_test, batch_size=128)

11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9481 - loss: 0.2932 


In [24]:
# Loading the saved model
model = tf.keras.models.load_model(model_save_path)

In [25]:
# Inference test
predict_result = model.predict(np.array([X_test[0]]))
print(np.squeeze(predict_result))
print(np.argmax(np.squeeze(predict_result)))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
[9.6637189e-02 7.4688756e-01 1.3554507e-01 1.0956612e-06 1.9928480e-02
 1.0005775e-03]
1


# Convert to model for Tensorflow-Lite

In [26]:
# Save as a model dedicated to inference
model.save(model_save_path, include_optimizer=False)

In [27]:
# Transform model (quantization)

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quantized_model = converter.convert()

open(tflite_save_path, 'wb').write(tflite_quantized_model)

INFO:tensorflow:Assets written to: C:\Users\narte\AppData\Local\Temp\tmpexsouyjp\assets


INFO:tensorflow:Assets written to: C:\Users\narte\AppData\Local\Temp\tmpexsouyjp\assets


Saved artifact at 'C:\Users\narte\AppData\Local\Temp\tmpexsouyjp'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 42), dtype=tf.float32, name='input_layer_1')
Output Type:
  TensorSpec(shape=(None, 6), dtype=tf.float32, name=None)
Captures:
  2286560071696: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2286560075536: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2286560073232: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2286560073424: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2286560074960: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2286560073616: TensorSpec(shape=(), dtype=tf.resource, name=None)


6628

# Inference test

In [28]:
interpreter = tf.lite.Interpreter(model_path=tflite_save_path)
interpreter.allocate_tensors()

In [29]:
# Get I / O tensor
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [30]:
interpreter.set_tensor(input_details[0]['index'], np.array([X_test[0]]))

In [31]:
%%time
# Inference implementation
interpreter.invoke()
tflite_results = interpreter.get_tensor(output_details[0]['index'])

CPU times: total: 0 ns
Wall time: 0 ns


In [32]:
print(np.squeeze(tflite_results))
print(np.argmax(np.squeeze(tflite_results)))

[9.6637167e-02 7.4688756e-01 1.3554512e-01 1.0956591e-06 1.9928465e-02
 1.0005770e-03]
1
